<a href="https://colab.research.google.com/github/Rihan-Fei/Canny-with-3-thresholds-V.S.-HED/blob/main/Canny%20manually%20implemented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
import math
from PIL import ImageFilter
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
im = Image.open('path to your image')
im_gray = im.convert('L')   #convert to gray iages

plt.imshow(im_gray, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
im_gray = im_gray.filter(ImageFilter.SMOOTH) #Filter

In [ ]:

im_array = np.array(im_gray)

#Initialize gradient array
grad = np.zeros((im_array.shape[0], im_array.shape[1], 2), dtype=np.float32)

#Calculate gradient
for j in range(1, im_array.shape[0] - 1):
    for i in range(1, im_array.shape[1] - 1):
        #X-direction gradient
        grad_x = im_array[j + 1, i + 1] + im_array[j - 1, i + 1] + 2 * im_array[j, i + 1] - \
                  im_array[j - 1, i - 1] - im_array[j + 1, i - 1] - 2 * im_array[j, i - 1]

        #y-direction gradient
        grad_y = im_array[j + 1, i - 1] + im_array[j + 1, i + 1] + 2 * im_array[j + 1, i] - \
                  im_array[j - 1, i - 1] - im_array[j - 1, i + 1] - 2 * im_array[j - 1, i]

        #Normalized gradient
        grad_x = grad_x / 4.0
        grad_y = grad_y / 4.0

        #Composite gradient modulus
        grad[j, i, 0] = min(255, math.sqrt(grad_x ** 2 + grad_y ** 2))

        #Gradient direction
        if grad_x == 0 and grad_y == 0:
            grad[j, i, 1] = 2  # y-direction
            theta = math.atan2(grad_y, grad_x)
            if abs(theta) < math.pi / 8:
                grad[j, i, 1] = 0  # x-direction
            elif theta > 0:
                if abs(theta) < math.pi * 3 / 8:
                    grad[j, i, 1] = 1  # 45 degrees
                else:
                    grad[j, i, 1] = 2  # y-direction
            else:
                if abs(theta) < math.pi * 3 / 8:
                    grad[j, i, 1] = 3  # 135 degrees
                else:
                    grad[j, i, 1] = 2  # y-direction


gradient_image = np.zeros_like(im_array, dtype=np.uint8)
for i in range(im_array.shape[0]):
    for j in range(im_array.shape[1]):
        gradient_image[i, j] = grad[i, j, 0]

plt.figure(2)
plt.imshow(gradient_image, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
img2 = np.array(im_gray)

#Dual threshold detection, modify by yourself
low = 50
high = 180

#Create a blank image to store the results
img2_result = np.zeros_like(img2)

for i in range(img2.shape[0]):  # 行
    for j in range(img2.shape[1]):  # 列
        if img2[i, j] < low:
            img2_result[i, j] = 0
        elif img2[i, j] > high:
            img2_result[i, j] = 255

#Display the image after dual threshold detection
plt.figure(4)
plt.imshow(img2_result, cmap='gray')
plt.axis('off')
plt.show()

#Suppressing isolated weak edges
img3 = img2_result.copy()
listx = [-1, 0, 1]
listy = [-1, 0, 1]

for j in range(1, img2.shape[0] - 1):
    for i in range(1, img2.shape[1] - 1):
        flag = 1
        for dx in range(len(listx)):
            for dy in range(len(listy)):
                xi = i + listx[dx]
                yj = j + listy[dy]
                if img2_result[yj, xi] == 255:
                    flag = 0
                    break
            if not flag:
                break
        if flag:
            img3[j, i] = 0


plt.figure(5)
plt.imshow(img3, cmap='gray')
plt.axis('off')
plt.show()